In [1]:
import numpy as np
from tqdm import tqdm
from copy import deepcopy

In [2]:
# user defined parameters

pool = 6 # The pool the armor to be augmented belongs to
init_budget = 10 # The initial budget of the armor piece

rng_sd = 0 # Seed for rng
rng = np.random.default_rng(rng_sd)

base_armor = {'Defense': 0, 'Fire res': 0, 'Water res': 0, 'Thunder res': 0,
              'Ice res': 0, 'Dragon res': 0, 'Slot': [4, 0, 0],
              'Skill': {'Weakness Exploit': 2, 'Offensive Guard': 2, 'Flinch Free': 3}} # Ingot Greaves X
target_armor = {'Slot': [4, 2, 1], 'Skill': {'Weakness Exploit': 2, 'Offensive Guard': 2}}

In [3]:
data = []
with open(f'./data/aug_val_pool_{pool}.csv', 'r') as f:
    header = f.readline()
    line = f.readline()
    while line:
        entries = line.split(',')
        data.append((entries[0], int(entries[1]), int(entries[2]), int(entries[3])))
        line = f.readline()

1. Takes the first defense aug in aug pool for the 7th augment.
2. Doesn't take any defense aug when the remaining budget doesn't match the cost of any single defense aug.

In [4]:
def get_qurio_aug(init_budget):
    def allowed_upgrade(roll, upgrade):
        if roll == 0:
            # first roll must be defense
            if upgrade[:-1] == 'Defense':
                return True
            else:
                return False
        elif roll == 1:
            # second roll must be skill
            if upgrade[:-1] == 'Skill':
                return True
            else:
                return False
        else:
            return True

    def clean_up_def():
        # drop all defense augments if they sum up to 0
        # return True if augs have been cleaned up, else return False
        def_ind = []
        total_def = 0
        for ind, (upgrade, lv, cost) in enumerate(total_augs):
            if upgrade[:-1] == 'Defense':
                def_ind.append(ind)
                total_def += lv
        if total_def == 0 and len(def_ind) > 0:
            for ind in def_ind[::-1]:
                del total_augs[ind]
            return True
        else:
            return False

    def remaining_budget():
        return init_budget - sum([cost for upgrade, lv, cost in total_augs])

    total_augs = []
    for roll in range(50):
        if len(total_augs) == 6:
            # take a defense boost according to the remaining budget
            for upgrade, lv, prob, cost in data:
                if upgrade[:-1] == 'Defense' and cost == remaining_budget():
                    total_augs.append((upgrade, lv, cost))
            if clean_up_def():
                continue
            else:
                break

        allowed_augs = []
        for upgrade, lv, prob, cost in data:
            if allowed_upgrade(roll, upgrade):
                allowed_augs.extend([(upgrade, lv, cost)] * prob)
        chosen_ind = rng.integers(0, high=len(allowed_augs))
        chosen_upgrade, chosen_lv, chosen_cost = allowed_augs[chosen_ind]
        if chosen_cost > remaining_budget():
            continue
        total_augs.append((chosen_upgrade, chosen_lv, chosen_cost))
        if clean_up_def():
            continue

        if remaining_budget() == 0:
            break

    return total_augs

In [5]:
skills_dict = {(3*i+3): [] for i in range(5)}
with open("./data/skill_cost.csv", 'r') as f:
    lines = f.readlines()
    for line in lines[1:]:
        entries = line.strip().split(',')
        cost = int(entries[1])
        name = entries[2]
        skills_dict[cost].append(name)

1. Each skill has the same chance of being chosen for `'Skill-'`, regardless of their points.
2. Prviously augmented skills can be chosen for `'Skill-'`.

In [6]:
def get_augmented_armor(base_armor):
    # get augmented armor based on the base armor
    
    armor = deepcopy(base_armor)
    qurio_augs = get_qurio_aug(init_budget)
    
    for upgrade, level, cost in qurio_augs:
        if upgrade == 'Skill+':
            chosen_ind = rng.integers(0, high=len(skills_dict[cost]))
            chosen_skill = skills_dict[cost][chosen_ind]
            if chosen_skill in armor['Skill']:
                armor['Skill'][chosen_skill] += 1
            else:
                armor['Skill'][chosen_skill] = 1
        elif upgrade == 'Skill-':
            chosen_ind = rng.integers(0, high=len(armor['Skill']))
            chosen_skill = list(armor['Skill'].keys())[chosen_ind]
            if armor['Skill'][chosen_skill] == 1:
                del armor['Skill'][chosen_skill]
            else:
                armor['Skill'][chosen_skill] -= 1
        elif upgrade == 'Slot+':
            for _ in range(level):
                if 0 in armor['Slot']:
                    armor['Slot'][armor['Slot'].index(0)] = 1
                else:
                    for ind in range(3):
                        if armor['Slot'][ind] < 4:
                            armor['Slot'][ind] += 1
                            break
        else:
            armor[upgrade[:-1]] += level
    
    return armor

In [7]:
def check_armor(armor, target_armor):
    # check if the augmented armor meets target requirement

    output = True
    for key, val in target_armor.items():
        if not output:
            break
        if key == 'Slot':
            for x, y in zip(armor[key], val):
                if x < y:
                    output = False
        elif key == 'Skill':
            for skill, level in val.items():
                if skill not in armor[key]:
                    output = False
                elif armor[key][skill] < level:
                    output = False
        else:
            if armor[key] < val:
                output = False
                
    return output

In [8]:
# Monte Carlo
num_rep = 100000
result = []

for _ in tqdm(range(num_rep)):
    result.append(check_armor(get_augmented_armor(base_armor), target_armor))

prob = sum(result) / len(result)
print(prob)

100%|██████████| 100000/100000 [00:18<00:00, 5415.52it/s]

0.00109


In [9]:
total_rolls = 200
1 - (1 - prob)**total_rolls # chance of getting one in total_rolls consecutive rolls

0.19597016027488257

In [10]:
conf = 0.8 # how unlucky you feel: from very lucky to very unlucky is 0 to 1
np.log(1-conf) / np.log(1-prob) # max num of rolls to get the target armor in conf probability

1475.7436782909192